In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [5]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.png').take(263)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.png').take(263)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.png').take(263)

In [6]:
# def preprocess(file_path):
#     byte_img = tf.io.read_file(file_path)
#     img = tf.io.decode_png(byte_img)    
#     # img = tf.image.resize(img, (128,64))
#     img = img / 255
#     img = img[:,:,0]
#     return img

In [7]:
# preprocessing
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_png(byte_img)    
    # img = tf.image.resize(img, (128,64))
    img = img / 255
    img = img[:,:,0]
    # print(type(img))
    # img = tf.repeat([img], repeats=[3], axis=0)
    # img = tf.transpose(img, perm=[1, 2, 0])
    # img = img.transpose(2,0,1)
    # print(tf.shape(img))
    return img

In [8]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [9]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [10]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [11]:
len(data)

525

In [12]:
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
print(len(test_data))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)
print(len(test_data))

157
10


In [ ]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [ ]:
print(len(test_input))
print(len(test_val))
print(y_true.shape)

In [13]:
# distancing/differencing layer
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [14]:
model = tf.keras.models.load_model('siamesemodel_vgg_custom_200+300+200+300_SGD_1e-2_m0.9_binary.h5', custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [ ]:
plt.imshow(test_input[2], cmap="gray")

In [ ]:
plt.imshow(test_val[2], cmap="gray")

In [ ]:
y_true[1]

In [ ]:
# testing for 1 image
y_hat = model.predict([np.expand_dims(test_input[1], axis=0), np.expand_dims(test_val[1], axis=0)])
y_hat

In [ ]:
# testing for all test images
pr_arr = []
f1_arr = []
acc_arr = []

for (test_input, test_val, y_true) in test_data.as_numpy_iterator():
    y_hat = model.predict([test_input, test_val])

    r = Recall()
    r.update_state(y_true, y_hat)
    recall_score = r.result().numpy()

    p = Precision()
    p.update_state(y_true, y_hat)
    precision_score = p.result().numpy()

    f1_score = (2 * precision_score * recall_score) / (precision_score + recall_score)

    a = BinaryAccuracy()
    a.update_state(y_true, y_hat)
    accuracy_score = a.result().numpy()

    # print(f'Recall: {recall_score}, Precision: {precision_score}')

    pr_arr.append((recall_score, precision_score))
    f1_arr.append(f1_score)
    acc_arr.append(accuracy_score)

f1_sum, acc_sum = 0, 0

for i in range(len(f1_arr)):
    f1_sum += f1_arr[i]
    acc_sum += acc_arr[i]

f1_avg = f1_sum / len(f1_arr)
acc_avg = acc_sum / len(acc_arr)

print(f'Avg Accuracy: {acc_avg}, Avg F1 Score: {f1_avg}')

In [ ]:
f1_arr

In [ ]:
acc_arr

In [ ]:
pr_arr

In [ ]:
# f1_sum = 0
# for i in range(len(f1_arr)-1):
#     f1_sum += f1_arr[i]
#     print(f1_arr[i])

# print(f'Sum: {f1_sum}')
# f1_avg = f1_sum / (len(f1_arr)-1)
# print(f'Avg: {f1_avg}')

In [ ]:
# from PIL import Image

In [ ]:
# glaucoma_img = Image.open(r'D:\Projects\college\Glaucoma Generation - Final Year\fsl trial v2 - glaucoma\data\anchor\POAG-000008-2009-02-03-OD.png')
# glaucoma_np = np.asarray(glaucoma_img)

In [ ]:
# normal_img = Image.open(r'D:\Projects\college\Glaucoma Generation - Final Year\fsl trial v2 - glaucoma\data\negative\Normal-000002-2009-10-28-OD.png')
# normal_np = np.asarray(normal_img)

In [ ]:
# def preprocess(file_path):
#     byte_img = tf.io.read_file(file_path)
#     img = tf.io.decode_png(byte_img)    
#     # img = tf.image.resize(img, (128,64))
#     img = img / 255
#     img = img[:,:,0]
#     return img

In [ ]:
# glaucoma_np = preprocess(r'D:\Projects\college\Glaucoma Generation - Final Year\fsl trial v2 - glaucoma\data\anchor\POAG-000008-2009-02-03-OD.png')
# normal_np = preprocess(r'D:\Projects\college\Glaucoma Generation - Final Year\fsl trial v2 - glaucoma\data\positive\POAG-001815-2009-03-31-OD.png')

In [ ]:
# glaucoma_np = np.expand_dims(glaucoma_np, axis=0)
# normal_np = np.expand_dims(normal_np, axis=0)
# glaucoma_np.shape

In [ ]:
# print(glaucoma_np.shape)
# print(normal_np.shape)

In [ ]:
# y_hat = model.predict([glaucoma_np, normal_np])
# y_hat

In [ ]:
# [1 if prediction > 0.5 else 0 for prediction in y_hat]